### CNN Interface for Gluon simplified in a similar manner to symbolic interface

Transition from symbolic MXNet to Gluon simplified

In [1]:
import mxnet as mx
from mxnet import nd, autograd
from mxnet import gluon
import numpy as np
mx.random.seed(1)

class BaseCNNClassifier(mx.gluon.Block):
    def __init__(self, ctx):
        super(BaseCNNClassifier, self).__init__()
        self.ctx = ctx
        self.net = None
        
    #@override
    def build_model(self, convs, num_fc, num_classes):
        '''
        Default activation is relu
        '''
        # convs = [(channel, kernel_sz, pool_siz)triplets *N]
        cnn_layers = gluon.nn.HybridSequential(prefix='')
        for ch, k_sz, p_sz in convs:
            cnn_layers.add(gluon.nn.Conv2D(channels=ch, kernel_size=k_sz, activation='relu'))
            cnn_layers.add(gluon.nn.MaxPool2D(pool_size=p_sz, strides=2)) # strides fixed for now
            
        net = gluon.nn.HybridSequential()
        with net.name_scope():
            net.add(cnn_layers)
            # Flatten and apply fully connected layers
            net.add(gluon.nn.Flatten())
            net.add(gluon.nn.Dense(num_fc, activation="relu"))
            net.add(gluon.nn.Dense(num_classes))

        # speed up execution with hybridization
        net.hybridize()
        self.net = net
    
    def forward(self):
        pass

    def compile_model(self, loss=None, optimizer='sgd', lr=1E-3, init_mg=2.24):
        print self.net
        self.net.collect_params().initialize(mx.init.Xavier(magnitude=init_mg), ctx=self.ctx)
        self.loss = gluon.loss.SoftmaxCrossEntropyLoss() if loss is None else loss
        self.optimizer = mx.gluon.Trainer(self.net.collect_params(), 
                                          optimizer, {'learning_rate': lr})
    
    def evaluate_accuracy(self, data_iterator):
        acc = mx.metric.Accuracy()
        for i, (data, label) in enumerate(data_iterator):
            data = data.as_in_context(ctx)
            label = label.as_in_context(ctx)
            output = self.net(data)
            predictions = nd.argmax(output, axis=1)
            acc.update(preds=predictions, labels=label)
        return acc.get()[1]
    
    def fit(self, train_data, test_data, epochs):
        
        smoothing_constant = .01
        ctx = self.ctx
        
        for e in range(epochs):
            for i, (data, label) in enumerate(train_data):
                data = data.as_in_context(ctx)
                label = label.as_in_context(ctx)
                with autograd.record(train_mode=True):
                    output = self.net(data)
                    loss = self.loss(output, label)
                loss.backward()
                self.optimizer.step(data.shape[0])

                ##########################
                #  Keep a moving average of the losses
                ##########################
                curr_loss = nd.mean(loss).asscalar()
                moving_loss = (curr_loss if ((i == 0) and (e == 0)) 
                               else (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss)

            test_accuracy = self.evaluate_accuracy(test_data)
            train_accuracy = self.evaluate_accuracy(train_data)
            print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (e, moving_loss, train_accuracy, test_accuracy))       

In [5]:
batch_size = 64
num_inputs = 784
num_outputs = 10

def transform(data, label):
    return nd.transpose(data.astype(np.float32), (2,0,1))/255, label.astype(np.float32)

train_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=True, transform=transform),
                                      batch_size, shuffle=True)
test_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=False, transform=transform),
                                     batch_size, shuffle=False)

In [7]:
train_data.__dict__

{'_batch_sampler': <mxnet.gluon.data.sampler.BatchSampler at 0x10b07bd10>,
 '_dataset': <mxnet.gluon.data.vision.MNIST at 0x10b07ba50>}

In [19]:
num_fc = 512
num_classes = 10 #num_outputs
convs = [(20,5,2), (50,5,2)]

ctx = mx.cpu()
cnn = BaseCNNClassifier(ctx)
cnn.build_model(convs, num_fc, num_classes)
cnn.compile_model(optimizer='adam')
cnn.fit(train_data, test_data, epochs=10)

HybridSequential(
  (0): HybridSequential(
    (0): Conv2D(20, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
    (2): Conv2D(50, kernel_size=(5, 5), stride=(1, 1))
    (3): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  )
  (1): Flatten
  (2): Dense(512, Activation(relu))
  (3): Dense(10, linear)
)
Epoch 0. Loss: 0.0550339012977, Train_acc 0.987516666667, Test_acc 0.9882
Epoch 1. Loss: 0.0430203394256, Train_acc 0.991333333333, Test_acc 0.9901
Epoch 2. Loss: 0.0261364316174, Train_acc 0.994883333333, Test_acc 0.991
Epoch 3. Loss: 0.0220908905365, Train_acc 0.996566666667, Test_acc 0.9921
Epoch 4. Loss: 0.0174940514529, Train_acc 0.996866666667, Test_acc 0.9919
Epoch 5. Loss: 0.0138277958493, Train_acc 0.997283333333, Test_acc 0.9901
Epoch 6. Loss: 0.0104377611519, Train_acc 0.998066666667, Test_acc 0.9921
Epoch 7. Loss: 0.0120992395219, Train_acc 0.998816666667, Test_acc 0.9929
Epoch 8. Loss:

In [ ]:
# Do CIFAR

https://github.com/ilkarman/DeepLearningFrameworks/blob/master/Gluon_CIFAR.ipynb

In [ ]:
# Lets try CIFAR now

In [3]:
%%time
from utils import *

# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)


Done.
Extracting files...
Done.
Preparing train set...
Preparing test set...
Done.
((50000, 3, 32, 32), (10000, 3, 32, 32), (50000,), (10000,))
(dtype('float32'), dtype('float32'), dtype('int32'), dtype('int32'))
CPU times: user 5.27 s, sys: 3.81 s, total: 9.08 s
Wall time: 1min 34s


In [ ]:
X_train, y_train = self.from_numpy(X_train, y_train)
train_loader = mx.gluon.data.DataLoader(mx.gluon.data.ArrayDataset(X_train, y_train),
                                                batch_size=batch_size, shuffle=True)


In [9]:
def transformer(data, label):
    data = mx.image.imresize(data, 224, 224)
    data = mx.nd.transpose(data, (2,0,1))
    data = data.astype(np.float32)
    return data, label

train_data = gluon.data.DataLoader(
    gluon.data.vision.CIFAR10('./data', train=True, transform=transformer),
    batch_size=batch_size, shuffle=True, last_batch='discard')

test_data = gluon.data.DataLoader(
    gluon.data.vision.CIFAR10('./data', train=False, transform=transformer),
    batch_size=batch_size, shuffle=False, last_batch='discard')

In [10]:
num_fc = 512
num_classes = 10 #num_outputs
convs = [(50,3,2), (50,3,2), (100,3,2), (100,3,2)]

ctx = mx.cpu()
cnn = BaseCNNClassifier(ctx)
cnn.build_model(convs, num_fc, num_classes)
cnn.compile_model(optimizer='adam')
cnn.fit(train_data, test_data, epochs=5)

HybridSequential(
  (0): HybridSequential(
    (0): Conv2D(50, kernel_size=(3, 3), stride=(1, 1))
    (1): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
    (2): Conv2D(50, kernel_size=(3, 3), stride=(1, 1))
    (3): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
    (4): Conv2D(100, kernel_size=(3, 3), stride=(1, 1))
    (5): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
    (6): Conv2D(100, kernel_size=(3, 3), stride=(1, 1))
    (7): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  )
  (1): Flatten
  (2): Dense(512, Activation(relu))
  (3): Dense(10, linear)
)


KeyboardInterrupt: 

In [ ]:
def yield_batch(X, y, batchsize=64, shuffle=False):
    
    def shuffle_data(X, y):
        s = np.arange(len(X))
        np.random.shuffle(s)
        X = X[s]
        y = y[s]
        return X, y

    assert len(X) == len(y)
    if shuffle:
        X, y = shuffle_data(X, y)
    # Only complete batches are submitted
    for i in range(len(X)//batchsize):
        yield X[i*batchsize:(i+1)*batchsize], y[i*batchsize:(i+1)*batchsize]